In [ ]:
import pandas as pd
import re

# 파일 읽기 및 전처리
파일 읽기 및 디버깅을 위한 아이디값 부여<br>
kapt와 sapt(서울시공공주택정보)를 join한 테이블은 apt라고 이름붙였음

In [ ]:
apt_excel_path = '../input/kapt_sapt_병합됨.xlsx'
apt_csv_path = '../input/kapt_sapt_병합됨.csv'

건축물대장_excel_path = '../input/건축물대장_주용도필터_디자인추가.xlsx'
건축물대장_csv_path = '../input/건축물대장.csv'

In [ ]:
# apt = pd.read_excel(apt_excel_path)
# apt.to_csv(apt_csv_path, index=False)

# # 건축물대장 = pd.read_excel(건축물대장_excel_path)
# # 건축물대장.to_csv(건축물대장_csv_path, index=False)

In [ ]:
apt = pd.read_csv(apt_csv_path)
print(f"apt: {len(apt)}")

건축물대장 = pd.read_csv(건축물대장_csv_path)
print(f"건축물대장: {len(건축물대장)}")

# id 생성
apt.insert(0, 'apt_id', range(1, len(apt) + 1))
건축물대장.insert(0, '건축물대장_id', range(1, len(건축물대장) + 1))

# 법정동 주소 파싱

In [ ]:
def apt_parse_법정동주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)    
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)     # 주소값이 없을 경우에 널값 리턴
    
    시도 = pd.NA
    군구 = pd.NA
    동리= pd.NA
    번지= pd.NA
    상세= pd.NA
    중복주소= pd.NA
    addr_rest = addr
    
    # 주소 파싱
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+)\s*(.*)", addr_rest)
    if match_groups:
        동리 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)(?:번지)?-?\s*(.*)", addr_rest)
    if match_groups:
        번지 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 동리, 번지, 상세, 중복주소)

apt[
    ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)', '법정동주소(중복주소)']
] = apt['법정동주소'].apply(
    lambda x: pd.Series(apt_parse_법정동주소(x))
)   

In [ ]:
def 건축물대장_parse_대지위치(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    동리= pd.NA
    번지= pd.NA
    상세= pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+)\s*(.*)", addr_rest)
    if match_groups:
        동리 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)(?:번지)?-?\s*(.*)", addr_rest)
    if match_groups:
        번지 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    if addr_rest:
        상세 = addr_rest
        
    return (시도, 군구, 동리, 번지, 상세)

건축물대장[
    ['법정동주소(시도)', '법정동주소(군구)', '법정동주소(동리)','법정동주소(번지)', '법정동주소(상세)']
] = 건축물대장['대지위치'].apply(
    lambda addr: pd.Series(건축물대장_parse_대지위치(addr))
)   

# 도로명 주소 파싱

In [ ]:
def apt_parse_도로명주소(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    도로명 = pd.NA
    건물번호 = pd.NA
    중복주소 = pd.NA
    상세 = pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+(?:로|길))\s*(.*)", addr_rest)
    if match_groups:
        도로명 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)\s*(.*)", addr_rest)
    if match_groups:
        건물번호 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    if addr_rest:
        if addr_rest.startswith(','):
            중복주소 = addr_rest.lstrip(',').strip()
        else:
            상세 = addr_rest
        
    return (시도, 군구, 도로명, 건물번호, 상세, 중복주소)

apt[
    ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)', '도로명주소(중복주소)']
] = apt['도로명주소'].apply(
    lambda x: pd.Series(apt_parse_도로명주소(x))
)

In [ ]:
def 건축물대장_parse_도로명대지위치(addr):
    if addr is None or pd.isna(addr): return(pd.NA, pd.NA, pd.NA, pd.NA)
    addr = addr.strip()
    if not addr: return(pd.NA, pd.NA, pd.NA, pd.NA)
    
    시도 = pd.NA
    군구 = pd.NA
    도로명 = pd.NA
    건물번호 = pd.NA
    상세 = pd.NA
    addr_rest = addr
    
    match_groups = re.match(r"(.+?시)\s*(.*)", addr)
    if match_groups:
        시도 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    match_groups = re.match(r"(.+?구)\s*(.*)", addr_rest)
    if match_groups:
        군구 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\S+(?:로|길))\s*(.*)", addr_rest)
    if match_groups:
        도로명 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
        
    match_groups = re.match(r"(\d+(?:-\d+)?)\s*(.*)", addr_rest)
    if match_groups:
        건물번호 = match_groups.group(1)
        addr_rest = match_groups.group(2).strip()
    
    addr_rest = re.sub(r"\([^)]*\)", "", addr_rest).strip()
    if addr_rest:
        상세 = addr_rest
        
    return (시도, 군구, 도로명, 건물번호, 상세)

건축물대장[
    ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)', '도로명주소(상세)']
] = 건축물대장['도로명대지위치'].apply(
    lambda x: pd.Series(건축물대장_parse_도로명대지위치(x))
)

# 병합

In [ ]:
# 매칭의 기준이 되는 컬럼 리스트
법정동주소_cols = ['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)']
도로명주소_cols = ['도로명주소(시도)', '도로명주소(군구)', '도로명주소(도로명)', '도로명주소(건물번호)']

In [ ]:
# 매칭된 데이터의 주소키는 apt의 값을 따르도록하는 함수
protected_cols = 법정동주소_cols + 도로명주소_cols
suffix = '_건축물대장'

def restore_건축물대장_cols(df):
    건축물대장_cols = [c for c in df.columns if c.endswith(suffix)]     # 출처가 건축물대장인 컬럼명 리스트
    for col in 건축물대장_cols:
        original_column_name = col.replace(suffix, '')  
        if original_column_name in protected_cols:
            continue
        else:
            df[original_column_name] = df[col]  
    return df.drop(columns=건축물대장_cols)                             # 출처가 건축물대장인 주소키 컬럼 제거

In [ ]:
# apt, 건축물대장 merge
def merge_apt_with_건축물대장(
    apt_df,     # apt
    건축물대장_df, # 건축물대장
    on_list     # 매칭의 기준이 되는 컬럼 리스트
    ):
    # LEFT JOIN
    merged = apt_df.merge(
        건축물대장_df.dropna(subset=on_list, how='all'),
        on = on_list,
        how = 'left',
        suffixes = ('','_건축물대장'),  # 출처가 건축물대장인 컬럼명 뒤에 '_건축물대장'을 붙임
        indicator = True
    )
    merged = restore_건축물대장_cols(merged)    # 중복컬럼(주소키) 병합 및 suffixes 제거
    
    matched = ( # LEFT JOIN 결과물 중 양쪽 매칭에 성공한 것
    merged.loc[merged['_merge'] == 'both']
    .drop(columns=['_merge'])
    .copy()
    )
    
    unmatched_apt = ( # 매칭에 실패한 apt 데이터
        merged.loc[merged['_merge'] == 'left_only']
        .drop(columns=['_merge'])
        .copy()
    )
    
    return matched, unmatched_apt

In [ ]:
# 우선적으로는 총괄표제부만 매칭에 참여
총괄_건축물대장 = 건축물대장[건축물대장['그룹 종류'] == 0]
print(f"총괄_건축물대장 : {len(총괄_건축물대장)}")
print(f"apt : {len(apt)}")


# 법정동주소로 매칭
matched_on_법정동주소, unmatched_apt = merge_apt_with_건축물대장(apt, 총괄_건축물대장, 법정동주소_cols)
matched_on_법정동주소['매칭방식'] = '총괄,법정동'
print(f"matched_on_법정동주소 : {len(matched_on_법정동주소)}")


# 도로명주소로 매칭
matched_on_도로명주소, unmatched_apt = merge_apt_with_건축물대장(unmatched_apt, 총괄_건축물대장, 도로명주소_cols)
matched_on_도로명주소['매칭방식']='총괄,도로명'
print(f"matched_on_도로명주소 : {len(matched_on_도로명주소)}")


matched = pd.concat([matched_on_도로명주소,matched_on_법정동주소], ignore_index=True)                   # 매칭 성공한 것
merged = pd.concat([matched_on_도로명주소, matched_on_법정동주소, unmatched_apt], ignore_index=True)    # 매칭 성공한 것 + 매칭 실패한 것
print(f"unmatched : {len(unmatched_apt)}")
print(f"matched : {len(matched)}")
print(f"merged : {len(merged)}")

In [ ]:
# 총괄표제부를 가지지 않는 일반표제부만 매칭에 참여
일반_건축물대장 = 건축물대장[건축물대장['그룹 종류'] == 2] 


# 법정동주소로 매칭
matched_on_법정동주소, unmatched_apt = merge_apt_with_건축물대장(unmatched_apt, 일반_건축물대장, 법정동주소_cols)
matched_on_법정동주소['매칭방식'] = '일반,법정동'
print(f"matched_on_법정동주소 : {len(matched_on_법정동주소)}")


# 도로명주소로 매칭
matched_on_도로명주소, unmatched_apt = merge_apt_with_건축물대장(unmatched_apt, 일반_건축물대장, 도로명주소_cols)
matched_on_도로명주소['매칭방식']='일반,도로명'
print(f"matched_on_도로명주소 : {len(matched_on_도로명주소)}")


print(f"unmatched : {len(unmatched_apt)}")

######## 일반표제부는 결과 테이블에 참여시키지 않음 ########
# matched = pd.concat(
#     [matched, matched_on_법정동주소, matched_on_도로명주소],
#     ignore_index=True
# )
# print(f"final: {len(matched)}")

# 디자인 추가

In [ ]:
final_df = matched.copy()

def color_매칭방식(val):
    colors = {
        '총괄,법정동': 'background-color: #04248B',# 진한 파랑
        '총괄,도로명': 'background-color: #4D75F9',# 연한 파랑
        '일반,법정동': 'background-color: #1D5A0C',# 진한 초록
        '일반,도로명': 'background-color: #8AEB6F',# 연한 초록
    }
    return colors.get(val, '')

법정동_cols = ['법정동주소(시도)','법정동주소(군구)','법정동주소(동리)','법정동주소(번지)','법정동주소(상세)']
도로명_cols = ['도로명주소(시도)','도로명주소(군구)','도로명주소(도로명)','도로명주소(건물번호)','도로명주소(상세)']

dup_법정동 = final_df.duplicated(subset=법정동_cols, keep=False)
dup_도로명 = final_df.duplicated(subset=도로명_cols, keep=False)
dup_pk = final_df['건축물대장_id'].notna() & final_df['건축물대장_id'].duplicated(keep=False)
dup_id = final_df['apt_id'].notna() & final_df['apt_id'].duplicated(keep=False)
def is_all_null(row, cols):
    return row[cols].isna().all()

def highlight_all(row):
    styles = [''] * len(row.index)

    # 매칭방식 컬럼 색칠
    if '매칭방식' in row.index:
        j = list(row.index).index('매칭방식')
        styles[j] = color_매칭방식(row['매칭방식'])

    # 중복인 경우 빨강
    if dup_법정동.loc[row.name] and not is_all_null(row, 법정동_cols):
        for i, col in enumerate(row.index):
            if col in 법정동_cols:
                styles[i] = 'background-color: #ffcccc'

    if dup_도로명.loc[row.name] and not is_all_null(row, 도로명_cols):
        for i, col in enumerate(row.index):
            if col in 도로명_cols:
                styles[i] = 'background-color: #ffcccc'
                
    if dup_pk.loc[row.name]:
        i = list(row.index).index('건축물대장_id')
        styles[i] = 'background-color: #ffcccc'
        
    if dup_id.loc[row.name]:
        i = list(row.index).index('apt_id')
        styles[i] = 'background-color: #ffcccc'

    return styles

final_styled = final_df.style.apply(highlight_all, axis=1)

# 결과

In [ ]:
final_styled.to_excel("../output/apt_건축물대장_병합_디자인추가.xlsx", index=False)
matched.to_excel("../output/apt_건축물대장_병합.xlsx", index=False)
unmatched_apt.to_excel("../output/apt_병합실패.xlsx", index=False)


used_건축물대장_ids = (
    matched['건축물대장_id'].unique()
)
unmatched_건축물대장 = 건축물대장[
    ~건축물대장['건축물대장_id'].isin(used_건축물대장_ids)
].copy()
unmatched_건축물대장.to_excel("../output/건축물대장_병합실패.xlsx", index=False)

In [ ]:
matched.to_excel("../output/apt_건축물대장_병합.xlsx", index=False)